In [37]:
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt

# Data Exploratory

In [38]:
df = pd.read_csv('stock fundamental data.csv')
len(df)


50560

In [39]:
df.head()


,gvkey,tic,fyear,curcd,act,ap,at,ceq,che,cogs,...,spi,txp,txt,xad,xido,xint,xrd,xsga,costat,mkvalt
0,1004,AIR,2009.0,USD,863.429,114.906,1501.042,746.906,79.370,1065.902,...,-4.302,3.263,20.986,NaN,0.000,26.832,NaN,151.904,A,777.8348
1,1004,AIR,2010.0,USD,913.985,185.096,1703.727,835.845,57.433,1408.071,...,-1.536,0.000,35.364,NaN,-3.313,30.670,NaN,171.399,A,1049.8206
2,1004,AIR,2011.0,USD,1063.272,201.405,2195.653,864.649,67.720,1662.408,...,-13.864,0.000,25.480,NaN,0.000,37.772,NaN,189.397,A,485.2897
3,1004,AIR,2012.0,USD,1033.700,149.300,2136.900,918.600,75.300,1714.500,...,-21.100,0.000,26.700,NaN,0.000,41.600,NaN,207.400,A,790.0029
4,1004,AIR,2013.0,USD,1116.900,171.100,2199.500,999.500,89.200,1581.400,...,0.000,0.000,32.100,NaN,0.000,42.000,NaN,197.600,A,961.3080


In [40]:
print("The shape of the data is:", df.shape)
print(df.info)

The shape of the data is: (50560, 35)
<bound method DataFrame.info of         gvkey  tic   fyear curcd       act       ap        at       ceq  \
0        1004  AIR  2009.0   USD   863.429  114.906  1501.042   746.906   
1        1004  AIR  2010.0   USD   913.985  185.096  1703.727   835.845   
2        1004  AIR  2011.0   USD  1063.272  201.405  2195.653   864.649   
3        1004  AIR  2012.0   USD  1033.700  149.300  2136.900   918.600   
4        1004  AIR  2013.0   USD  1116.900  171.100  2199.500   999.500   
...       ...  ...     ...   ...       ...      ...       ...       ...   
50555  328795  ACA  2015.0   USD       NaN      NaN       NaN       NaN   
50556  328795  ACA  2016.0   USD   419.900   48.900  1526.300  1341.800   
50557  328795  ACA  2017.0   USD   428.800   56.000  1602.500  1407.900   
50558  328795  ACA  2018.0   USD   667.000   86.200  2172.200  1684.500   
50559  328795  ACA  2019.0   USD   757.200   90.000  2302.500  1790.400   

           che      cogs  ...

In [41]:
#find the ticker num 
tic_num = df['tic'].unique()
print("The number of tickers are:", len(tic_num), "tickers") 

The number of tickers are: 4932 tickers


In [42]:
#change columns names 
df = df.rename(columns={'act': 'Total Current Assets', 'ap': 'Accounts Payable', 'ceq': 'Total Ordinary Equity',
                        "cogs": "Cost of Goods Sold", "dlc":"Total Debt in Current Liabilities", "dlcch": "Current Debt Changes", 
                        "dltt": "Total Long-Term Debt", 'dp':'Depreciation and Amortization',
                        'dvc':'Dividends Common/Ordinary', 'ib':'Income Before Extraordinary Items', 
                        'intan': 'Total Intangible Assets','invt': 'Total Inventories','ivao':'Investments and Advances',
                        'lct':'Total Current Liabilities', 'lt':'Total Liabilities', 
                        'nopio':'Nonoperating Income (expense)', 'ppent':'Total Property, Plant&Equipment',
                        'spi':'ecial Items', 'txp':'Income Taxes Payable', 'txt':'Total Income Taxes', 
                        'xad':'Advertising Expense', 'xido':'Extraordinary Items and Discontinued Operations', 
                        'xint':'Total Interest and Related Expense', 'xrd':'Research and Development Expense', 
                        'xsga':'Selling,General and Administrative Expense', 'rect':'Total Receivables', 'at':'Total Assets', 'che':'Cash and Short-Term Investments'})
df.head()

,gvkey,tic,fyear,curcd,Total Current Assets,Accounts Payable,Total Assets,Total Ordinary Equity,Cash and Short-Term Investments,Cost of Goods Sold,...,ecial Items,Income Taxes Payable,Total Income Taxes,Advertising Expense,Extraordinary Items and Discontinued Operations,Total Interest and Related Expense,Research and Development Expense,"Selling,General and Administrative Expense",costat,mkvalt
0,1004,AIR,2009.0,USD,863.429,114.906,1501.042,746.906,79.370,1065.902,...,-4.302,3.263,20.986,NaN,0.000,26.832,NaN,151.904,A,777.8348
1,1004,AIR,2010.0,USD,913.985,185.096,1703.727,835.845,57.433,1408.071,...,-1.536,0.000,35.364,NaN,-3.313,30.670,NaN,171.399,A,1049.8206
2,1004,AIR,2011.0,USD,1063.272,201.405,2195.653,864.649,67.720,1662.408,...,-13.864,0.000,25.480,NaN,0.000,37.772,NaN,189.397,A,485.2897
3,1004,AIR,2012.0,USD,1033.700,149.300,2136.900,918.600,75.300,1714.500,...,-21.100,0.000,26.700,NaN,0.000,41.600,NaN,207.400,A,790.0029
4,1004,AIR,2013.0,USD,1116.900,171.100,2199.500,999.500,89.200,1581.400,...,0.000,0.000,32.100,NaN,0.000,42.000,NaN,197.600,A,961.3080


In [43]:
#check for null data 
df.isnull().sum()

gvkey                                                  0
tic                                                    0
fyear                                                 51
curcd                                                 51
Total Current Assets                               21788
Accounts Payable                                   10863
Total Assets                                        6722
Total Ordinary Equity                               6843
Cash and Short-Term Investments                    13777
Cost of Goods Sold                                 13868
Total Debt in Current Liabilities                   7086
Current Debt Changes                               31465
Total Long-Term Debt                                6884
Depreciation and Amortization                       9347
Dividends Common/Ordinary                           7149
Income Before Extraordinary Items                   6821
Total Intangible Assets                             7427
Total Inventories              

In [44]:
#df drop the null value 
df = df.ffill(limit=1)
df = df.fillna(0)

In [45]:
#Evaluate the company financially stability, through out the years. Using: Total Current Assets/Total Current Liabilities 
#classifield stable if > 1 and unstable if <1 
df['ratio'] = df['Total Current Assets']/df['Total Current Liabilities']

#create a new columsn to output fin stables 
#if fi_stable > 1:
df['good stable status'] = (df['ratio'] > 1)
df.head()


,gvkey,tic,fyear,curcd,Total Current Assets,Accounts Payable,Total Assets,Total Ordinary Equity,Cash and Short-Term Investments,Cost of Goods Sold,...,Total Income Taxes,Advertising Expense,Extraordinary Items and Discontinued Operations,Total Interest and Related Expense,Research and Development Expense,"Selling,General and Administrative Expense",costat,mkvalt,ratio,good stable status
0,1004,AIR,2009.0,USD,863.429,114.906,1501.042,746.906,79.370,1065.902,...,20.986,0.0,0.000,26.832,0.0,151.904,A,777.8348,2.652216,True
1,1004,AIR,2010.0,USD,913.985,185.096,1703.727,835.845,57.433,1408.071,...,35.364,0.0,-3.313,30.670,0.0,171.399,A,1049.8206,2.197027,True
2,1004,AIR,2011.0,USD,1063.272,201.405,2195.653,864.649,67.720,1662.408,...,25.480,0.0,0.000,37.772,0.0,189.397,A,485.2897,2.246859,True
3,1004,AIR,2012.0,USD,1033.700,149.300,2136.900,918.600,75.300,1714.500,...,26.700,0.0,0.000,41.600,0.0,207.400,A,790.0029,2.657326,True
4,1004,AIR,2013.0,USD,1116.900,171.100,2199.500,999.500,89.200,1581.400,...,32.100,0.0,0.000,42.000,0.0,197.600,A,961.3080,2.777667,True


In [46]:
 ###THIS BLOCK OF CODE IS CREDITED TO YAAKOV MY TEAMMATE
#create column 'future mkvalt' that holds the market value of the next year
condition1 = df['tic'].shift(-1) == df['tic']
condition2 = df['fyear'].shift(-1) == df['fyear'] + 1
df['future_mkvalt'] = np.where(condition1 & condition2, df['mkvalt'].shift(-1), np.nan)

#percent change is the percent change in market value from one year to the next
df['percent_change'] = (df['future_mkvalt']-df['mkvalt'])/df['mkvalt']

#if percent change is greater than 20%, classify as undervalued
df['undervalued'] = (df['percent_change'] > 0.10) 


In [47]:
#Expense 
df['Expense'] = df['Research and Development Expense'] + df['Selling,General and Administrative Expense'] + df['Total Interest and Related Expense'] + df['Advertising Expense'] + df['Nonoperating Income (expense)']

In [48]:
#drop unecessary columns 
expense_list = ['Research and Development Expense','Selling,General and Administrative Expense','Total Interest and Related Expense','Advertising Expense','Nonoperating Income (expense)']
df = df.drop(columns = expense_list)
df = df.drop(columns = ["gvkey","Cost of Goods Sold","ecial Items","Extraordinary Items and Discontinued Operations","costat","Income Taxes Payable","Total Income Taxes","curcd"])

In [49]:
#drop NaN, and finite 
df = df.dropna(subset=["future_mkvalt", "percent_change"], how='all') 

In [60]:
df = df.replace([np.inf, -np.inf], np.nan)
df = df.fillna(0)

In [61]:
df.isnull().sum()

tic                                  0
fyear                                0
Total Current Assets                 0
Accounts Payable                     0
Total Assets                         0
Total Ordinary Equity                0
Cash and Short-Term Investments      0
Total Debt in Current Liabilities    0
Current Debt Changes                 0
Total Long-Term Debt                 0
Depreciation and Amortization        0
Dividends Common/Ordinary            0
Income Before Extraordinary Items    0
Total Intangible Assets              0
Total Inventories                    0
Investments and Advances             0
Total Current Liabilities            0
Total Liabilities                    0
Total Property, Plant&Equipment      0
Total Receivables                    0
sale                                 0
mkvalt                               0
ratio                                0
good stable status                   0
future_mkvalt                        0
percent_change           

In [62]:
from sklearn import svm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, classification_report
from sklearn import metrics 

# KNN Classifier 

In [63]:
#get training data and testing data 
X = df.drop(columns=['tic', 'fyear', 'future_mkvalt', 'percent_change', 'undervalued'])
y = df['undervalued']


In [64]:
SEED = 77
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=SEED)

In [71]:
knn = KNeighborsClassifier()
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
score =  knn.score(X_test, y_test)
print(score)
#classification_report 
print(classification_report(y_test, y_pred))
# classifier got around 62% of classes right

0.6211193022531409
              precision    recall  f1-score   support

       False       0.69      0.73      0.71      6068
        True       0.49      0.44      0.46      3563

    accuracy                           0.62      9631
   macro avg       0.59      0.58      0.58      9631
weighted avg       0.61      0.62      0.62      9631



# Naives Bayes

In [76]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
GB = GaussianNB()
GB.fit(X_train, y_train)
y_pred = GB.predict(X_test)
accuracy_score = accuracy_score(y_test,y_pred)
print("accuracy_score: ", accuracy_score)

accuracy_score:  0.3911328003322604


In [77]:
#confusion matrix 
cm = confusion_matrix(y_test, y_pred)
print('Confusion matrix\n\n', cm)

Confusion matrix

 [[ 347 5721]
 [ 143 3420]]


In [78]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

       False       0.71      0.06      0.11      6068
        True       0.37      0.96      0.54      3563

    accuracy                           0.39      9631
   macro avg       0.54      0.51      0.32      9631
weighted avg       0.58      0.39      0.27      9631

